# Imports

In [1]:
# export
from collections import namedtuple, defaultdict
import os
import re
from nbdev_rewrite.imports import *

# Helpers

In [2]:
# hide
def run_tests(cases, func, verbose=False):
    """Run test cases by passing a list of tuples (test name, input to func, expected result)"""
    nr_correct = 0
    for i, (n, c, r) in enumerate(cases):
        if verbose: print(f'({i + 1} / {len(cases)}) TEST {n}:')
        try:
            res = func(c)
            assert res == r, f'TEST FAILED WITH RESULT: {res}\nEXPECTED: {r}'
            nr_correct += (res == r)
            if verbose: print(f'TEST RESULT: SUCCESS\n')
        except Exception as e:
            if verbose: print(f'TEST FAILED WITH EXCEPTION:\n{e}\n')
            # raise e
    print('--------------- ALL TESTS COMPLETED ---------------')
    print(f'{nr_correct} / {len(cases)} Correct')

# Init

In [3]:
create_config('nbdev_rewrite', 'flpeters', nbs_path='.')

In [4]:
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# Notebook Loading

In [26]:
#export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r', encoding='utf8') as f: return nbformat.reads(f.read(), as_version=4)

In [36]:
test_nb = read_nb('00_export.ipynb')

In [28]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [29]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.6'},
 'toc': {'base_numbering': 1,
  'nav_menu': {},
  'number_sections': True,
  'sideBar': True,
  'skip_h1_title': False,
  'title_cell': 'Table of Contents',
  'title_sidebar': 'Contents',
  'toc_cell': False,
  'toc_position': {},
  'toc_section_display': True,
  'toc_window_display': False}}

In [30]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.4'

In [31]:
test_nb['cells'][0]

{'cell_type': 'markdown', 'metadata': {}, 'source': '# Imports'}

In [32]:
len(test_nb['cells'])

70

# Keyword Comments

## extract comments

`iter_comments()` is used to find and extract all comments from a code block.  
It's main purpose is to avoid matching on "comments" that are actually just part of a string, and not real python comments. One example would be: 

    """
    # export
    """
A naive parser would see the literal "#" and match that line. In reality however, this code snippet is a string, and might be e.g. part of a test suit (which is how this bug was found in the first place), and not really meant to be exported.  
View https://docs.python.org/3/reference/lexical_analysis.html#strings for more info

In [12]:
# export
# TODO(florian): Only look for 0 indent comments?
def iter_comments(src:str, pure_comments_only:bool=True, line_limit=None):
    """Detect all comments in a piece of code, excluding those that are a part of a string."""
    in_lstr = in_sstr = False
    count, quote = 1, ''
    for i, line in enumerate(src.splitlines()[:line_limit]):
        is_pure, escape, prev_c = True, False, '\n'
        for j, c in enumerate(line):
            # we can't break as soon as not is_pure, because we have to detect if a multiline string beginns
            if is_pure and (not (c.isspace() or c == '#')): is_pure = False
            if (in_sstr or in_lstr):
                # assert (in_sstr and not in_lstr) or (in_lstr and not in_sstr)
                if escape: count = 0
                else:
                    if (c == quote):
                        count = ((count + 1) if (c == prev_c) else 1)
                        if in_sstr: in_sstr = False
                        elif (in_lstr and (count == 3)): count, in_lstr = 0, False
                escape = False if escape else (c == '\\')
            else:                    
                if (c == '#'):
                    if (pure_comments_only and is_pure): yield (line, (i, j))
                    elif (not pure_comments_only):       yield (line[j:], (i, j))
                    break
                elif c == "'" or c == '"':
                    count = ((count + 1) if (c == prev_c) else 1)
                    if count == 1: in_sstr = True
                    elif count == 3: count, in_lstr = 0, True
                    else: raise SyntaxError(f'Unexpected quote repetition count: {count} Should be either 1 or 3')
                    quote = c
            prev_c = c

In [13]:
def test_iter_comments(src): return list(iter_comments(src, True))
test_strings = [
("trippe quote(''')", """'''
# string
'''""", []),
('tripple quote(""")', '''"""
#string
"""''', []),
('single quote(")', '"\
\n#string\n\
"', []),
("single quote(')", "'\
\n#string\n\
'", []),
("simple comment", """
#comment
""", [('#comment', (1, 0))]),
("comment sandwich", """
'this is a string'
# this is a comment
'another string , but between is an actual comment'
""", [('# this is a comment', (2, 0))]),
("tricky case 2", """
  a #non-pure comment
'''
#string
'''
####comment""", [('####comment', (5, 0))]),
("end of string quote", """
'''
'
# still part of the string
'
'''
""", []),
("single end of string escape", """
'\\'\\\n#str\\\n\\''
""", []),
("weird escape sequence", """
'\\\n\\''
""", []),
("long end of string escape", """
'''
# string
\\'''
# string
'''
""", []),
("raw string escape", """
r'''
# string
\\'''
# string
'''
""", []),
("multiple strings", """
'''a''''''b'''
""", []),
]
run_tests(test_strings, test_iter_comments, verbose=False)

--------------- ALL TESTS COMPLETED ---------------
13 / 13 Correct


## find keywords

In [14]:
# export
class KeywordParser:
    def __init__(self, *init_keywords):
        self.parsers = {}
        for kw in init_keywords: self.parsers[kw] = self._create_parser(kw)

    def _create_parser(self, keyword):
        # TODO: decide on the syntax
        # TODO: Should there be any whitespace allowed before special comments?
        # TODO: Should more than one "#" be allowed for special comments?
        pattern = fr"""
        ^              # start of line, since MULTILINE is passed
        \s*            # any amount of whitespace
        \#+\s*          # literal "#", then any amount of whitespace
        {keyword}(.*)  # keyword followed by arbitrary symbols (except new line)
        $              # end of line, since MULTILINE is passed
        """
        return re.compile(pattern, re.IGNORECASE | re.MULTILINE | re.VERBOSE)

    def __getitem__(self, key):
        if key in self.parsers: return self.parsers[key]
        else:
            parser = self._create_parser(key)
            self.parsers[key] = parser
            return parser
        
#     def _search_remove(self, key, text):
#         print('WARNING: _search_remove() DOESN\'T WORK YET')
#         # TODO: This function is supposed to remove the keyword comment from the input
#         # TODO: detect_comments() has to be modified to allow for the positions to be returned
#         parser = self[key]
#         text, locations = detect_comments(text)
#         for comment, l in zip(text, locations):
#             res = parser.search(comment)
#             if res: return res, l

## parse keyword options

In [15]:
# export
OptionsTuple = namedtuple(typename='Options',
                          field_names=['internal', 'export_target'],
                          defaults=[False, None])

In [16]:
# export
_re_legacy_options = re.compile(r'^(i)?\s*([a-zA-Z0-9]+\S*|)\s*$')
def legacy_parse_options(options:str) -> OptionsTuple:
    res = _re_legacy_options.search(options)
    if res:
        internal, export_target = res.groups()
        return OptionsTuple(internal=(internal == 'i'),
                            export_target=(os.path.sep.join(export_target.split('.')) if export_target else None))
    else: return None

In [17]:
# export
def parse_options(options:str, legacy:bool=True) -> OptionsTuple:
    if (options is None) or (options == '') or (options.isspace()): return OptionsTuple()
    else:
        if legacy:
            res = legacy_parse_options(options)
            if res: return res
            else: pass # Fall through to different parsing scheme
        # TODO: New Syntax for specifying keyword options
        raise NotImplementedError('this branch of parse_options() is not implemented yet.')

## determine exports

In [18]:
def remove_comment(source:str, loc_line:int, loc_char:int=None) -> str:
    lines = source.splitlines()
    if loc_char is None: lines.pop(loc_line)
    else: lines[loc_line] = lines[loc_line][:loc_char] # pass loc_char to only remove part of the line and keep the rest
    return '\n'.join(lines)

In [19]:
t = """first line
second line
third line"""

print(remove_comment(t, 1))
print('')
print(remove_comment(t, 1, 6))

first line
third line

first line
second
third line


In [106]:
# export
keyword_parser = KeywordParser()
kw_export, kw_hide = keyword_parser['export'], keyword_parser['hide']
def find_exports(cells:list, default:str, code_only:bool=True) -> list:
    """check for each cell if it's supposed to be exported and aggregate cell content together with export options"""
    exports = []
    for i, cell in enumerate(cells):
        if code_only and (cell.cell_type != 'code'): continue
        else:
            source = cell.source
            for comment, (loc_line, loc_char) in iter_comments(source):
                res = kw_export.search(comment)
                if res:
                    options = parse_options(res.groups()[0])
                    assert options.export_target or default, f'Cell nr.{i} doesn\'t have an export target, \
                                                                and no default is specified:\n{source}'
                    if not options.export_target: options = options._replace(export_target=default)
                    source = remove_comment(source, loc_line, None)
                    source = re.sub(r'\s+$', '', source, flags=re.MULTILINE)
                    exports.append((source, options))
                    continue
                if kw_hide.search(comment): break
    return exports

## Tests

In [78]:
test_nb['cells'][0].keys()

dict_keys(['cell_type', 'metadata', 'source'])

In [79]:
test_nb['cells'][0]

{'cell_type': 'markdown', 'metadata': {}, 'source': '# Imports'}

In [80]:
len(find_exports(test_nb['cells'], 'export', code_only=True))

20

In [107]:
test_strings = [
("trippe quote(''')", """'''
#export
'''""", []),
('tripple quote(""")', '''"""
#export
"""''', []),
('single quote(")', '"\
\n#export\n\
"', []),
("single quote(')", "'\
\n#export\n\
'", []),
("correct", """
#export
""", [('', OptionsTuple(internal=False, export_target='default'))]),
("tricky case 1", """
'this is a string'
#export
'this also, but between is an actual comment'
""", [("""
'this is a string'
'this also, but between is an actual comment'""",
OptionsTuple(internal=False, export_target='default'))]),
("tricky case 2", """
  a #export
'''
#export
'''
####export""", [("""
  a #export
'''
#export
'''""", OptionsTuple(internal=False, export_target='default'))]),
("tricky case 3", """
'''
'
# export
'
'''
""", []),
("tricky case 4", """
'''
\'
# export
\'
'''
""", []),
]
def test_find_exports(x):
    class y: pass
    y.cell_type, y.source = 'code', x
    return find_exports([y], default='default')
run_tests(test_strings, test_find_exports, verbose=False)

--------------- ALL TESTS COMPLETED ---------------
9 / 9 Correct


In [108]:
test_markup = [
('export', """
# export
""", [('', OptionsTuple(internal=False, export_target='default'))]),
('comment layout', """
#export
""", [('', OptionsTuple(internal=False, export_target='default'))]),
('export internal legacy', """
# exporti
""", [('', OptionsTuple(internal=True, export_target='default'))]),
('export internal legacy with target', """
# exporti some.module
""", [('', OptionsTuple(internal=True, export_target='some\\module'))]),
('export internal', """
# export -i
""", [('', OptionsTuple(internal=True, export_target='default'))]),
('export show source', """
# export -s
""", [('', OptionsTuple(internal=False, export_target='default'))]),
('export internal show', """
# export -i -s
""", [('', OptionsTuple(internal=True, export_target='default'))]),
('default empty', """

""", []),
('hide', """
# hide
""", []),
('multiple comments', """
# export
# hide
""", [('\n# hide', OptionsTuple(internal=False, export_target='default'))]),
('multiple comments other way', """
# hide
# export
""", []),
('multi comment same line', """
# export hide
""", [('', OptionsTuple(internal=False, export_target='hide'))]),
('multiple comments default_exp', """
# export
# default_exp
""", [('\n# default_exp', OptionsTuple(internal=False, export_target='default'))]),
]
run_tests(test_markup, test_find_exports, verbose=True)

(1 / 13) TEST export:
TEST RESULT: SUCCESS

(2 / 13) TEST comment layout:
TEST RESULT: SUCCESS

(3 / 13) TEST export internal legacy:
TEST RESULT: SUCCESS

(4 / 13) TEST export internal legacy with target:
TEST RESULT: SUCCESS

(5 / 13) TEST export internal:
TEST FAILED WITH EXCEPTION:
this branch of parse_options() is not implemented yet.

(6 / 13) TEST export show source:
TEST FAILED WITH EXCEPTION:
this branch of parse_options() is not implemented yet.

(7 / 13) TEST export internal show:
TEST FAILED WITH EXCEPTION:
this branch of parse_options() is not implemented yet.

(8 / 13) TEST default empty:
TEST RESULT: SUCCESS

(9 / 13) TEST hide:
TEST RESULT: SUCCESS

(10 / 13) TEST multiple comments:
TEST RESULT: SUCCESS

(11 / 13) TEST multiple comments other way:
TEST RESULT: SUCCESS

(12 / 13) TEST multi comment same line:
TEST RESULT: SUCCESS

(13 / 13) TEST multiple comments default_exp:
TEST RESULT: SUCCESS

--------------- ALL TESTS COMPLETED ---------------
10 / 13 Correct


# Names

In [43]:
# export
import ast
from ast import iter_fields, AST
import _ast

This part is using pythons builtin ast module to parse code to be exported into an abstract syntax tree, from which the set of variable-, function-, and classnames is extracted.  
All names found, that are not private (prefixed with a single underscore), are later added to the `__all__` in the exported file.  
It also parses the special keyword variable `_all_` and adds all the names assigned to it in a list, tuple, set, or directly to `__all__`

In [44]:
# export
def unwrap_attr(node:_ast.Attribute) -> str:
    if isinstance(node.value, _ast.Attribute): return '.'.join((unwrap_attr(node.value), node.attr))
    else: return '.'.join((node.value.id, node.attr))

In [45]:
# export
def update_recursive(node, names):
    """inplace, recursive update of names"""
    if   isinstance(node, _ast.Name)      : names.append(node.id)
    elif isinstance(node, _ast.Starred)   : names.append(node.value.id)
    elif isinstance(node, _ast.Attribute) : names.append(unwrap_attr(node))
    elif isinstance(node, (_ast.List, _ast.Tuple)):
        for x in node.elts: update_recursive(x, names)
    elif isinstance(node, list):
        for x in node: update_recursive(x, names)
    else: raise SyntaxError(f'Can\'t resolve {node} to name, unknown type')

In [46]:
# export
def update_from_all_(node, names): # TODO(florian): should all of these cases be handled?
    """inplace, recursive update of names, by parsing the right side of a _all_ variable"""
    if isinstance(node, _ast.Str): names.add(node.s)
    elif isinstance(node, _ast.Name): names.add(node.id)
    elif isinstance(node, _ast.Attribute): names.add(unwrap_attr(node))
    elif isinstance(node, (_ast.List, _ast.Tuple, _ast.Set)):
        for x in node.elts: update_from_all_(x, names)
    elif isinstance(node, _ast.Starred): raise SyntaxError(f'Starred expression *{node.value.id} not allowed in _all_')
    else: raise SyntaxError(f'Can\'t resolve {node} to name, unknown type')

In [47]:
# export
def not_private(name): return not (name.startswith('_') and (not name.startswith('__')))

In [48]:
# export
def add_names(node, names):
    tmp_names = list()
    update_recursive(node.targets, tmp_names)
    for name in tmp_names:
        if not_private(name): names.add(name)
        # NOTE: cases below can only use private variable names
        elif name == '_all_':
            assert len(tmp_names) == 1, 'reserved keyword _all_ can only be used in simple assignments'
            update_from_all_(node.value, names)

In [49]:
# export
def find_names(code:str) -> list:
    try: tree = ast.parse(code) # expensive # TODO: make this use the python version specified in config
    except Exception as e: raise e # TODO: Logging
    else:
        names = set()
        for node in tree.body:
            if   isinstance(node, _ast.Assign): add_names(node, names)
            elif isinstance(node, (_ast.FunctionDef, _ast.ClassDef)) and not_private(node.name): names.add(node.name)
            else: pass
        return names

## Tests

In [50]:
test_assignment = [
('Default Assignment', """
a = 1
b = a
a = 2
""", {'a', 'b'}),
('Tuple unpacking', """
a, b = (1, 2)
""", {'a', 'b'}),
('unpacking to tuples and lists', """
(a, b) = (1, 2)
[a, b] = (1, 2)
""", {'a', 'b'}),
('unpacking to tuples and lists x2', """
([a], (b)) = (1, 2)
[[a, ((b))]] = (1, 2)
""", {'a', 'b'}),
('Multiple assignments', """
a = b = 2
""", {'a', 'b'}),
('List Deconstruction', """
head, *tail = [1,2,3,4,5]
""", {'head', 'tail'}),
('Private Variables', """
_a = 1
""", set()),
('Dunder Variables', """
__a = 1
""", {'__a'}),
('Attribues', """
a.b = 1
""", {'a.b'}),
('_all_ special keyword', """
_all_ = {'set', '__d'}
_all_ = ['var_a', var_b, a.b, 'c.d', _abc, '''x''']
""", {'set', '__d', 'var_a', 'var_b', 'a.b', 'c.d', '_abc', 'x'}),
]
run_tests(test_assignment, find_names, verbose=False)

--------------- ALL TESTS COMPLETED ---------------
10 / 10 Correct


In [51]:
test_funcdef = [
('Default function definition', """
def add(a, b):
    return a + b
""", {'add'}),
('Type Annotated function def', """
def calc(a:int, b:int) -> int:
    c:float = 2.0
    return (a + b) * c
""", {'calc'}),
('function decorators', """
@test1
@test2
def add(a, b):
    return a + b
""", {'add'}),
('@patch and more complex type annotations', """
@patch
def func (obj:(Class1, Class2), a:int)->int:
    pass
""", {'func'})
]
run_tests(test_funcdef, find_names)

--------------- ALL TESTS COMPLETED ---------------
4 / 4 Correct


In [52]:
test_classdef = [
('Default class definition', """
class Abc:
    pass
""", {'Abc'}),
('Default class def 2', """
class Abc():
    pass
""", {'Abc'}),
]
run_tests(test_classdef, find_names)

--------------- ALL TESTS COMPLETED ---------------
2 / 2 Correct


# Export

Used for keeping track of where each of the code cells are supposed to be exported to.  
The `key` is the filename, `code` is a list of strings (converted code cells) that will be added to the file, and `names` is the set of names of objects that are added to `__all__`.

In [53]:
# export
class ExportCache(defaultdict):
    def __init__(self, default_export:str=None):
        super(ExportCache, self).__init__(self._create_exp)
        self.tupletype = namedtuple(typename='export', field_names=['code', 'names'])
        if default_export is not None: self[default_export]
    
    def _create_exp(self): return self.tupletype(code=list(), names=set())
    
    def add_names(self, key:str, names:list): self[key].names.update(names)
            
    def add_code(self , key:str, code:str)  : self[key].code.append(code)

In [54]:
test_ec = ExportCache('default')
test_ec['test']
test_ec['abc']
assert 'default' in test_ec
assert 'test' in test_ec
assert 'abc' in test_ec
test_ec['test'].code.append('hi')
test_ec['test'].code.append('ho')
test_ec['test'].names.update({'nanana'})
assert test_ec['test'].code == ['hi', 'ho']
assert test_ec['test'].names == {'nanana'}
assert test_ec['abc'].code == []
assert test_ec['default'].names == set()
test_ec.add_names('default', ['xyz', 'jkl'])
test_ec.add_code('default', "print('Hello World!')")
test_ec.add_code('default', "print('Hello World!')")
assert test_ec['default'].names == {'xyz', 'jkl'}
assert test_ec['default'].code == ["print('Hello World!')", "print('Hello World!')"]
assert test_ec['abc'].names == set()
test_ec.pop('default')
assert not ('default' in test_ec)

In [55]:
# export
def find_default_export(cells:list) -> str:
    # search through all cells to find the default_exp keyword and return it's value.
    # syntax checking
    # maybe do some sanity checking
    default = 'export'
    return default

In [56]:
# export
def create_mod_file(orig_nbfname, targ_pyfname):
    # create the .py file in the correct folder, with a header saying where it was originally from
    pass

In [57]:
# export
def _notebook2script(fname=None, cells=None, silent=False, to_dict=False):
    """Convert a single notebook"""
    fname = Path(fname)
    assert (fname and not cells) or (not fname and cells)
    if not cells: # TODO(florian): this is temporarily used for testing, remove this
        nb = read_nb(fname)
        cells = nb['cells']
        
    sep = '\n' * (max(int(Config().get('cell_spacing', 1)), 0) + 1)
    
    default = find_default_export(cells)
    if default is None:
        print('WARNING: No default export target found! (should this crash, or see if each export has its own target?)')
        raise NotImplementedError('Not specifying a default export is not supported yet.')
    else:
        default = os.path.sep.join(default.split('.'))
        ec = ExportCache(default)
        # TODO(florian): create_mod_file(original_nbfile_path, target_pyfile_path) # args flipped in original code
        pass
    # TODO(florian): load _nbdev file and create a spec from it (no idea why this is needed)
    
    exports = find_exports(cells, default)
    for j, (code, options)  in enumerate(exports):
        # code = clean_code(code)
        # TODO: make imports of the current project relative in the output code
        i, e = options.internal, options.export_target
        if not i: ec.add_names(e, find_names(code))
        orig = (('Internal C' if i else '# C') if e==default else f'# Comes from {fname.name}, c') + 'ell\n'
        ec.add_code(e, (sep + orig + code))
        
    for e, s in ec.items():
        fname_out = Config().lib_path/f'{e}.py'
        nb_path = Config().nbs_path/f'{fname}'
        config_path = Config().config_file.parent
        rel_nb_path = os.path.relpath(nb_path, config_path).replace('\\', '/')
        fname_out.parent.mkdir(parents=True, exist_ok=True)
        warning = f'# AUTOGENERATED! DO NOT EDIT! File to edit: {rel_nb_path} (unless otherwise specified).'
        names = sep + "__all__ = ['" + "', '".join(s.names) + "']" # TODO(florian): add line breaks at regular intervals
        code  = ''.join(s.code)
        file_content = warning + names + code
        if e == default:
            with open(fname_out, 'w', encoding='utf8') as f: f.write(file_content)
        else: raise NotImplementedError('Exporting to a module other than the default is not supported yet.')
    # TODO(florian): add names to _nbdev index
    # TODO(florian): write code cell to file
    # TODO(florian): save _nbdev file
    return ec

In [58]:
# if default in exports:
#     write_file(exports.pop(default))
# return exports

In [59]:
# TODO(florian): initialize the library with __init__.py (and other stuff it needs?) if it doesn't already exist

In [60]:
ec = _notebook2script(
    fname='00_export.ipynb',
    # cells=test_nb['cells']
)

In [53]:
from nbdev_rewrite.export import _notebook2script

In [54]:
ec = _notebook2script(
    fname='00_export.ipynb',
    # cells=test_nb['cells']
)

In [47]:
from pprint import pprint

In [52]:
# TODO(florian): do a copy-rename swap, to prevent corruption of files in case of failure
# with safe_replace('export.py') as f:
#     f.write(data)

In [237]:
# export 
def notebook2script(fname=None, silent=False, to_dict=False):
    "Convert notebooks matching `fname` to modules"
    # initial checks
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    if fname is None:
        reset_nbdev_module()
        update_version()
        update_baseurl()
        files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    else: files = glob.glob(fname)
    d = collections.defaultdict(list) if to_dict else None
    for f in sorted(files): d = _notebook2script(f, silent=silent, to_dict=d)
    if to_dict: return d
    else: add_init(Config().lib_path)